<a href="https://colab.research.google.com/github/park-geun-hyeong/Kaggle_project/blob/main/kaggle_sentiment_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
%matplotlib inline

warnings.filterwarnings('ignore')

In [109]:
path="/content/drive/MyDrive/Kaggle/kaggle_bar of words/"

In [110]:
submission=pd.read_csv(path+"sampleSubmission.csv")
submission

,id,sentiment
0,12311_10,0
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,0
...,...,...
24995,2155_10,0
24996,59_10,0
24997,2531_1,0
24998,7772_8,0


In [111]:
import os

In [112]:
train=pd.read_csv(path+"labeledTrainData.tsv",delimiter='\t')
test=pd.read_csv(path+"testData.tsv",delimiter='\t')
unlable_train=pd.read_csv(path+"unlabeledTrainData.tsv",delimiter='\t',error_bad_lines=False)

b'Skipping line 43043: expected 2 fields, saw 3\n'


In [113]:
train

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...


In [114]:
test

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...
...,...,...
24995,2155_10,"Sony Pictures Classics, I'm looking at you! So..."
24996,59_10,I always felt that Ms. Merkerson had never got...
24997,2531_1,I was so disappointed in this movie. I am very...
24998,7772_8,"From the opening sequence, filled with black a..."


In [115]:
unlable_train

,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor Spoilers<br /><br />In New York, Joan Ba..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."
...,...,...
49993,18984_0,The original Man Eater by Joe D'Amato is somew...
49994,16433_0,When Home Box Office was in it's early days mo...
49995,16006_0,Griffin Dunne was born into a cultural family....
49996,40155_0,"Not a bad story, but the low budget rears its ..."


In [116]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer #run, ran ,runs -> run으로 바꿔주는 역할

In [117]:
import re

In [118]:
pip install nltk

In [119]:
import nltk

In [120]:
print(nltk.__version__)

3.2.5


In [121]:
sample=train['review'][0]

In [122]:
nltk.download('stopwords')
from nltk.corpus import stopwords
lematizer= WordNetLemmatizer()
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [123]:
eng_stopwords=stopwords.words('english')
eng_stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [124]:
def process_lemma(sentence):
    return [lematizer.lemmatize(word,'v') for word in sentence]

In [125]:
 def preprocessing(sentence):
    soup = BeautifulSoup(sentence,'html.parser')
    clean1 = re.sub('[^a-zA-Z]', ' ' ,soup.text)
    clean2 = clean1.lower()
    clean3 = [word for word in clean2.split() if word not in eng_stopwords]
    clean4 = process_lemma(clean3)
    return ' '.join(clean4)

In [126]:
preprocessing(sample)

'stuff go moment mj start listen music watch odd documentary watch wiz watch moonwalker maybe want get certain insight guy think really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember go see cinema originally release subtle message mj feel towards press also obvious message drug bad kay visually impressive course michael jackson unless remotely like mj anyway go hate find bore may call mj egotist consent make movie mj fan would say make fan true really nice actual feature film bite finally start minutes exclude smooth criminal sequence joe pesci convince psychopathic powerful drug lord want mj dead bad beyond mj overhear plan nah joe pesci character rant want people know supply drug etc dunno maybe hat mj music lot cool things like mj turn car robot whole speed demon sequence also director must patience saint come film kiddy bad sequence usually directors hate work one kid let alone whole bunch perform complex dance scene botto

In [127]:
all_review=pd.concat([train['review'],test['review'],unlable_train['review']])

In [128]:
all_review

0        With all this stuff going down at the moment w...
1        \The Classic War of the Worlds\" by Timothy Hi...
2        The film starts with a manager (Nicholas Bell)...
3        It must be assumed that those who praised this...
4        Superbly trashy and wondrously unpretentious 8...
                               ...                        
49993    The original Man Eater by Joe D'Amato is somew...
49994    When Home Box Office was in it's early days mo...
49995    Griffin Dunne was born into a cultural family....
49996    Not a bad story, but the low budget rears its ...
49997    This not-very-good mummy-alien flick does feat...
Name: review, Length: 99998, dtype: object

In [129]:
print(train.shape)
print(test.shape)
print(unlable_train.shape)

(25000, 3)
(25000, 2)
(49998, 2)


In [130]:
print(all_review.shape)

(99998,)


In [131]:
all_review_cleaned=all_review.apply(preprocessing)

In [132]:
all_review_cleaned.head()

0    stuff go moment mj start listen music watch od...
1    classic war worlds timothy hines entertain fil...
2    film start manager nicholas bell give welcome ...
3    must assume praise film greatest film opera ev...
4    superbly trashy wondrously unpretentious explo...
Name: review, dtype: object

In [133]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [134]:
tokenizer=Tokenizer(oov_token='<OOV>')

In [135]:
tokenizer.fit_on_texts(all_review_cleaned)

In [136]:
len(tokenizer.word_index)

126312

In [137]:
train_feature=all_review_cleaned[:len(train)]
test_feature=all_review_cleaned[25000:50000]

In [138]:
print(train_feature.shape)
print(test_feature.shape)

(25000,)
(25000,)


In [139]:
train_label=train['sentiment']
train_label.shape

(25000,)

In [140]:
train_seq = tokenizer.texts_to_sequences(train_feature)
test_seq = tokenizer.texts_to_sequences(test_feature)

In [141]:
train_seq[0]

[397,
 12,
 463,
 11594,
 83,
 931,
 127,
 13,
 895,
 507,
 13,
 21106,
 13,
 19437,
 179,
 46,
 8,
 639,
 2250,
 66,
 16,
 18,
 469,
 3273,
 179,
 5,
 188,
 643,
 2110,
 1155,
 19437,
 58,
 4431,
 58,
 258,
 2,
 240,
 12,
 7,
 349,
 1643,
 255,
 1145,
 550,
 11594,
 59,
 773,
 2039,
 29,
 471,
 550,
 593,
 26,
 4231,
 1924,
 1032,
 175,
 420,
 1453,
 782,
 2209,
 6,
 11594,
 459,
 12,
 613,
 37,
 170,
 116,
 146,
 11594,
 34889,
 9296,
 5,
 3,
 11594,
 109,
 15,
 25,
 5,
 109,
 198,
 18,
 253,
 727,
 258,
 2,
 114,
 339,
 83,
 141,
 7788,
 3475,
 1502,
 311,
 781,
 6909,
 526,
 9123,
 785,
 593,
 1370,
 46,
 11594,
 242,
 26,
 558,
 11594,
 9785,
 505,
 12451,
 781,
 6909,
 11,
 3763,
 46,
 27,
 24,
 2666,
 593,
 413,
 8743,
 179,
 724,
 11594,
 127,
 64,
 469,
 94,
 6,
 11594,
 90,
 419,
 1905,
 130,
 1523,
 2147,
 311,
 29,
 68,
 113,
 3929,
 3388,
 36,
 2,
 22089,
 26,
 311,
 516,
 843,
 613,
 43,
 4,
 129,
 152,
 518,
 130,
 630,
 890,
 1120,
 423,
 55,
 1131,
 107,
 3,
 27,
 6,
 

In [142]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [143]:
train_padded = pad_sequences(train_seq, maxlen=150, truncating='post',padding='post')
test_padded = pad_sequences(test_seq, maxlen=150, padding='post', truncating='post')

In [144]:
print(train_padded.shape)
print(test_padded.shape)

(25000, 150)
(25000, 150)


In [145]:
from sklearn.model_selection import train_test_split

In [146]:
train_label=train['sentiment']

In [147]:
x_train,x_valid,y_train,y_valid=train_test_split(train_padded, train_label, stratify=train_label, test_size=0.1,random_state=0)

In [148]:
embedding_dim=300

In [149]:
vocab_size=len(tokenizer.word_index)+1

In [150]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

In [151]:
model=Sequential([
     Embedding(vocab_size,embedding_dim, input_length=150),
     Bidirectional(LSTM(128, return_sequences=True)),
     Bidirectional(LSTM(128)),
     Dropout(0.25),
     Dense(32,activation='relu'),
     Dense(1, activation='sigmoid')

])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 300)          37893900  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 150, 256)          439296    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               394240    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 38,735,693
Trainable params: 38,735,693
Non-trainable params: 0
__________________________________________

In [152]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

In [153]:
checkpoint_path='tmp/checkpoint.ckpt'
checkpoint=ModelCheckpoint(filepath=checkpoint_path,save_best_only=True,save_weights_only=True,monitor='val_loss',verbose=1)

In [155]:
model.fit(x_train,y_train,validation_data=(x_valid,y_valid),epochs=20, batch_size=128, callbacks=[checkpoint])

Epoch 1/20
176/176 [==============================] - 571s 3s/step - loss: 0.4980 - acc: 0.7308 - val_loss: 0.3104 - val_acc: 0.8836

Epoch 00001: val_loss improved from inf to 0.31036, saving model to tmp/checkpoint.ckpt
Epoch 2/20
176/176 [==============================] - 575s 3s/step - loss: 0.1553 - acc: 0.9477 - val_loss: 0.3345 - val_acc: 0.8716

Epoch 00002: val_loss did not improve from 0.31036
Epoch 3/20
176/176 [==============================] - 570s 3s/step - loss: 0.0676 - acc: 0.9777 - val_loss: 0.4732 - val_acc: 0.8652

Epoch 00003: val_loss did not improve from 0.31036
Epoch 4/20
  3/176 [..............................] - ETA: 9:14 - loss: 0.0199 - acc: 0.9970

KeyboardInterrupt: ignored

In [156]:
model.load_weights(checkpoint_path)

In [157]:
model.evaluate(x_valid,y_valid)

79/79 [==============================] - 23s 295ms/step - loss: 0.3104 - acc: 0.8836


[0.31036344170570374, 0.8835999965667725]

In [158]:
pred = model.predict(test_padded)

In [159]:
pred[pred>=0.5] = 1
pred[pred<0.5] = 0 

In [160]:
pred

array([[1.],
       [0.],
       [1.],
       ...,
       [0.],
       [1.],
       [1.]], dtype=float32)

In [163]:
pred_df=pd.DataFrame(pred)
pred_df

,0
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
24995,1.0
24996,1.0
24997,0.0
24998,1.0


In [164]:
submission

,id,sentiment
0,12311_10,0
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,0
...,...,...
24995,2155_10,0
24996,59_10,0
24997,2531_1,0
24998,7772_8,0


In [165]:
submission.loc[:,'sentiment']=pred_df[0]
submission

,id,sentiment
0,12311_10,1.0
1,8348_2,0.0
2,5828_4,1.0
3,7186_2,1.0
4,12128_7,1.0
...,...,...
24995,2155_10,1.0
24996,59_10,1.0
24997,2531_1,0.0
24998,7772_8,1.0


In [166]:
submission['sentiment'] = submission['sentiment'].astype('int')
submission

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,1
3,7186_2,1
4,12128_7,1
...,...,...
24995,2155_10,1
24996,59_10,1
24997,2531_1,0
24998,7772_8,1


In [167]:
submission.to_csv("kaggle_nlp3.csv",index=False)